# Imports

## Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import re
from sklearn.impute import SimpleImputer

## Fonctions

In [ ]:
# Fontion qui crée un graphique plot avec des colonnes données pour un DataFrame
def graphs(dataframe, colonnes):

    # 1 couleur pour chaque indicateur
    colors = ["green", "red", "blue", "purple"]

    # liste des indicateurs pour faire 1 graphique par indicateur
    indicateurs = list(dataframe["Indicator Name"].unique())

    # nombre de graphique en fonction du nombre d'indicateur
    nb_graphs = len(indicateurs)

    # création de la figure avec une légère augmentation de la distance entre les graphiques
    fig, axes = plt.subplots(nb_graphs, 1, figsize=(10, 20))
    fig.subplots_adjust(hspace=0.5)

    # boucle sur les indicateurs
    for i, indic in enumerate(indicateurs):

        # récupération des données utiles
        frame = dataframe.loc[dataframe["Indicator Name"].isin(
            [indic])].set_index("Indicator Name")[colonnes].T

        ax1 = axes[i]

        # création du graphique
        frame.plot(ax=ax1,
                   title=indic,
                   grid=True,
                   legend=False,
                   xlabel="Années",
                   ylabel="Nombre d'étudiants",
                   xticks=range(0, len(colonnes), 5),
                   color=colors[i])
    
    # Affichage du graphique
    plt.show()


# Fontion qui crée un graphique plot pour un DataFrame transposé
def transposed_graphs(dataframe):

    # 1 couleur pour chaque indicateur
    colors = ["green", "red", "blue", "purple"]

    # nombre de graphique en fonction du nombre d'indicateur
    nb_graphs = len(dataframe.columns)

    # création de la figure avec une légère augmentation de la distance entre les graphiques
    fig, axes = plt.subplots(nb_graphs, 1, figsize=(10, 20))
    fig.subplots_adjust(hspace=0.5)

    # boucle sur les indicateurs
    for i, indic in enumerate(dataframe.columns):

        # récupération des données utiles
        frame = dataframe.loc[:, indic]

        ax1 = axes[i]

        # création du graphique
        frame.plot(ax=ax1,
                   title=indic,
                   grid=True,
                   legend=False,
                   xlabel="Années",
                   ylabel="Nombre d'étudiants",
                   xticks=range(0, len(frame), 5),
                   ylim=(0, int(frame.max()) * 1.1),
                   color=colors[i])

    # Affichage du graphique
    plt.show()


def distribution_graphs(dataframe):

    # 1 couleur pour chaque indicateur
    colors = ["green", "red", "blue", "purple"]

    # boucle sur les indicateurs
    for i, indic in enumerate(dataframe.columns):

        # récupération des données utiles
        frame = dataframe.loc[:, indic]

        # création du graphique
        frame.hist(figsize=(12, 8), grid=True, legend=False, color=colors[i])

        # Ajout des labels et titre
        plt.xlabel("Valeurs")
        plt.ylabel("Distribution")
        plt.title(f"Distribution des valeurs de {indic}")

        # Affichage du graphique
        plt.show()


# Fonction pour afficher un graphique avec 1 couleur différente pour les données interpolées
def interpolated_graphs(dataframe, interpolated):

    # 2 couleurs pour chaque indicateur
    colors = ["green", "red", "blue", "purple"]
    inter_colors = ["purple", "blue", "red", "green"]

    # nombre de graphique en fonction du nombre d'indicateur
    nb_graphs = len(dataframe.columns)

    # création de la figure avec une légère augmentation de la distance entre les graphiques
    fig, axes = plt.subplots(nb_graphs, 1, figsize=(10, 20))
    fig.subplots_adjust(hspace=0.5)

    # boucle sur les indicateurs
    for i, indic in enumerate(dataframe.columns):

        # récupération des données utiles
        frame = dataframe.loc[:, indic]
        inter_frame = interpolated.loc[:, indic]

        ax1 = axes[i]

        # création de la ligne des données interpolées
        inter_frame.plot(ax=ax1,
                         title=indic,
                         grid=True,
                         legend=False,
                         xlabel="Années",
                         ylabel="Nombre d'étudiants",
                         xticks=range(0, len(frame), 5),
                         ylim=(0, int(frame.max()) * 1.1),
                         color=colors[i])

        # Superposition de la ligne des données avant interpolation
        frame.plot(ax=ax1, color=inter_colors[i])

    # Affichage du graphique
    plt.show()


# Fonction qui ajoute 2 colonnes : 1 de la dernière valeur et 1 de la date de cette valeur
def last_line_value(dataframe, colonnes):

    # Récupération des données utiles
    frame = dataframe[colonnes]

    # Récupération du nombre de ligne et de colonnes
    line = frame.shape[0]
    col = frame.shape[1] - 1

    # Boucle sur les lignes
    for i in range(line):

        # Récupération de l'index
        index = frame.iloc[i].name

        # Boucle sur les colonnes en sens inverse
        for j in range(col, -1, -1):

            # Récupération de la valeur de la ligne et de la colonne
            value = frame.iloc[i, j]

            # Vérification si la valeur n'est pas vide
            if not pd.isna(value):

                # Ajout des valeurs et de la date
                dataframe.loc[index, "Last Value"] = value
                dataframe.loc[index,
                              "Last Value Date"] = pd.to_numeric(colonnes[j])

                #Sortie de la boucle
                break
            j += 1


# Fonction pour diviser une liste en plusieurs sous-liste
def diviser_liste(liste, taille):
    return [liste[i:i + taille] for i in range(0, len(liste), taille)]

## Données

In [ ]:
# Import des différents fichiers
data = pd.read_csv("../Data/EdStatsData.csv")
country_data = pd.read_csv("../Data/EdStatsCountry.csv")
country_series = pd.read_csv("../Data/EdStatsCountry-Series.csv")
foot_note = pd.read_csv("../Data/EdStatsFootNote.csv")
series = pd.read_csv("../Data/EdStatsSeries.csv")

# Liste des indicateurs choisis
indicateurs_precis = [
    "Enrolment in post-secondary non-tertiary education, both sexes (number)",
    "Enrolment in secondary education, both sexes (number)",
    "Enrolment in tertiary education, all programmes, both sexes (number)",
    "Internet users (per 100 people)"
]

pd.set_option('display.float_format', '{:,.2f}'.format)
pd.set_option('display.max_colwidth', None)

# Description des Données
## Shape

In [ ]:
# Création d'une liste avec les noms des différentes DataFrame
liste_nom = [
    "Données", "Pays", "Séries des pays", "Notes des séries", "Séries"
]

# Création d'une liste contenant les DataFrame
liste_dataframe = [data, country_data, country_series, foot_note, series]

# Affichage des formes de tous les DataFrame
print(*(f"{liste_nom[i]} : {liste_dataframe[i].shape}"
        for i in range(0, len(liste_nom))),
      sep="\n")

## Colonnes

In [ ]:
# Affichage des noms des colonnes de tous les DataFrame
print(
    *(f"Les colonnes de {liste_nom[i].lower()} : {liste_dataframe[i].columns}"
      for i in range(0, len(liste_nom))),
    sep="\n\n")

### Colonnes "Unnamed"

In [ ]:
# Afficahge de la Liste des colonnes avec un nom inconnu et le nombre de valeurs non nulles
print(*(liste_dataframe[i].loc[:, [
    col for col in liste_dataframe[i].columns if "unnamed" in col.lower()
]].notnull().sum() for i in range(0, len(liste_nom))),
      sep="\n\n")

### Suppression des colonnes "Unnamed"

In [ ]:
for frame in liste_dataframe:
    frame.drop(
        columns=[col for col in frame.columns if "unnamed" in col.lower()],
        inplace=True)

## Head des Dataframes

In [ ]:
data.head()

In [ ]:
series.head()

In [ ]:
country_data.head()

In [ ]:
country_series.head()

In [ ]:
foot_note.head()

# Exploration des donées
## Exploration du DataFrame Data
### Exploration globale

In [ ]:
data.info

In [ ]:
data.describe()

In [ ]:
# Vérification du nombre de lignes dupliquées
data.duplicated().sum()

In [ ]:
# Vérification et Affichage du nombre d'indicateurs par pays
data.groupby("Country Name")["Indicator Name"].count().unique()

In [ ]:
# Vérification et Affichage du nombre de pays par indicateurs
data.groupby("Indicator Name")["Country Name"].count().unique()

In [ ]:
# Création d'une liste avec tous les noms de pays
liste_countries = list(data["Country Name"].unique())

# Séparation de la liste pour les régions
liste_regions = liste_countries[0:liste_countries.index("World")+1]

# Séparation de la liste pour les pays
liste_pays = liste_countries[liste_countries.index("World")+1:]

# Affichage du nombre de régions et de pays
print(f"Nombre de régions : {len(liste_regions)} \n\nNombre de pays : {len(liste_pays)}")

In [ ]:
liste_code_pays = list(country_data["Country Code"].unique())
pays_manquant = data[~(data["Country Code"].isin(liste_code_pays))]
pays_manquant["Country Name"].unique()

In [ ]:
# Affichage du type des colonnes
print(data.loc[:, "Country Name":"Indicator Code"].dtypes)
print(data.loc[:, "1970":"2020"].dtypes)
print(data.loc[:, "2025":"2100"].dtypes)

### Visuels des données manquantes

In [ ]:
# Affichage des données d'un échantillon de 1000 lignes
msno.bar(data.sample(1000))

In [ ]:
# Affichage des données présentes par ligne pour un échantillon de 500 lignes 
msno.matrix(data.sample(500))

In [ ]:
# Calcul de la moyenne de données vides par colonnes
nb_colnull = data.isnull().mean() * 100

# Création de la figure avec 2 sous-graphiques
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Création du graphique des colonnes 1970 à 2017
nb_colnull.loc["1970":"2017"].plot(
    kind="bar",
    ax=ax1,
    title="Pourcentage de valeurs vides par colonne",
    xlabel="Nom des Colonnes",
    ylabel="Pourcentage")

# Ajout des valeurs en haut des colonnes
for i, value in enumerate(nb_colnull.loc["1970":"2017"]):
    ax1.annotate(f"{int(value)}", (i, value), ha="center", va="bottom")

# Création du deuxième graphique pour les autres colonnes
nb_colnull.loc["2020":].plot(kind="bar",
                             ax=ax2,
                             xlabel="Nom des Colonnes",
                             ylabel="Pourcentage")

# Ajout des valeurs en haut des colonnes
for i, value in enumerate(nb_colnull.loc["2020":]):
    ax2.annotate(f"{int(value)}", (i, value), ha="center", va="bottom")

# Ajuster l'espacement entre les sous-graphiques
plt.tight_layout()

# Affichage du graphique
plt.show()

In [ ]:
round(nb_colnull.mean(), 2)

In [ ]:
# Création de sous-listes avec 41 pays par liste
liste_countries = list(data["Country Name"].unique())
taille_sous_liste = 41
sous_listes = diviser_liste(liste_countries, taille_sous_liste)

# Boucle pour chaque sous-liste de pays
for liste in sous_listes:
    
    # Calcul de la moyenne de données vides par ligne
    nb_lignenull = data.loc[data["Country Name"].isin(liste)].set_index("Country Name").isnull().mean(axis=1) * 100

    # Groupement des données par pays.
    grouped_data = nb_lignenull.reset_index().groupby("Country Name").mean()

    # Créer une figure et une grille de sous-graphiques
    fig, ax = plt.subplots(figsize=(12, 8))

    # Tracé les barres horizontales
    grouped_data.plot.barh(ax=ax, color='b', legend=False, grid=True)

    # Ajout des titres et des étiquettes d'axe
    ax.set_title("Pourcentage de valeurs nulles par pays")
    ax.set_xlabel("Pourcentage de valeurs nulles")
    ax.set_ylabel("Pays")

    # Affichage du graphique
    plt.show()

## Exploration du DataFrame Country
### Exploration globale

In [ ]:
country_data.info

In [ ]:
# Vérification du nombre de lignes dupliquées
country_data.duplicated().sum()

In [ ]:
# Affichage du type des colonnes
country_data.dtypes

In [ ]:
country_data.loc[~(country_data["National accounts base year"].isnull()), "National accounts base year"]

### Visuels des données manquantes

In [ ]:
# Affichage des données d'un échantillon de 1000 lignes
msno.bar(country_data)

In [ ]:
# Affichage des données présentes par ligne pour un échantillon de 500 lignes 
msno.matrix(country_data)

In [ ]:
# Calcul de la moyenne de données vides par colonnes
nb_colnull = country_data.isnull().mean() * 100

# Création de la figure avec 2 sous-graphiques
fig, ax= plt.subplots(1, 1, figsize=(12, 8))

# Création du graphique des colonnes 1970 à 2017
nb_colnull.plot(
    kind="bar",
    title="Pourcentage de valeurs vides par colonne",
    xlabel="Nom des Colonnes",
    ylabel="Pourcentage",
    )

# Rotation des lalels de l'axe des abscisses
ax.set_xticklabels(ax.get_xticklabels(),
                   rotation=45,
                   horizontalalignment="right")

# Ajout des valeurs en haut des colonnes
for i, value in enumerate(nb_colnull):
    ax.annotate(f"{int(value)}", (i, value), ha="center", va="bottom")

# Ajuster l'espacement entre les sous-graphiques
plt.tight_layout()

# Afficher le graphique
plt.show()

In [ ]:
round(nb_colnull.mean(), 2)

In [ ]:
# Création d'une liste avec les noms des pays
liste_countries = list(country_data["Short Name"])
    
# Calcul de la moyenne de données vides par ligne
nb_lignenull = country_data.set_index("Short Name").isnull().mean(axis=1) * 100

# Créer une figure et une grille de sous-graphiques
fig, ax = plt.subplots(figsize=(12, 50))

# Tracé les barres horizontales
nb_lignenull.plot.barh(ax=ax, color='b', legend=False, grid=True)

#Ajout des valeurs au bout des barres
for i, value in enumerate(nb_lignenull):
    ax.annotate(f"{int(value)}", (value, i), ha="left", va="center")

# Ajout des titres et des étiquettes d'axe
ax.set_title("Pourcentage de valeurs nulles par pays")
ax.set_xlabel("Pourcentage de valeurs nulles")
ax.set_ylabel("Pays")

# Afficher le graphique
plt.show()

## Exploration du DataFrame Series
### Exploration globale

In [ ]:
series.info

In [ ]:
# Vérification du nombre de lignes dupliquées
series.duplicated().sum()

In [ ]:
# Vérification du tyoes des colonnes
series.dtypes

### Visuels des données manquantes

In [ ]:
# Affichage des données d'un échantillon de 1000 lignes
msno.bar(series.sample(1000))

In [ ]:
# Affichage des données présentes par ligne pour un échantillon de 500 lignes 
msno.matrix(series.sample(500))

In [ ]:
# Calcul de la moyenne de données vides par colonnes
nb_colnull = series.isnull().mean() * 100

# Création de la figure avec 2 sous-graphiques
fig, ax= plt.subplots(1, 1, figsize=(12, 8))

# Création du graphique des colonnes 1970 à 2017
nb_colnull.plot(
    kind="bar",
    title="Pourcentage de valeurs vides par colonne",
    xlabel="Nom des Colonnes",
    ylabel="Pourcentage",
    )

# Rotation des lalels de l'axe des abscisses
ax.set_xticklabels(ax.get_xticklabels(),
                   rotation=45,
                   horizontalalignment="right")

# Ajout des valeurs en haut des colonnes
for i, value in enumerate(nb_colnull):
    ax.annotate(f"{int(value)}", (i, value), ha="center", va="bottom")

# Ajuster l'espacement entre les sous-graphiques
plt.tight_layout()

# Afficher le graphique
plt.show()

In [ ]:
round(nb_colnull.mean(), 2)

In [ ]:
# Création de sous-listes avec 41 pays par liste
liste_series = list(series["Indicator Name"])
taille_sous_liste = 55
sous_listes = diviser_liste(liste_series, taille_sous_liste)

# Boucle pour chaque sous-liste de pays
for liste in sous_listes:
    
    # Calcul de la moyenne de données vides par ligne
    nb_lignenull = series.loc[series["Indicator Name"].isin(liste)].set_index("Indicator Name").isnull().mean(axis=1) * 100


    # Créer une figure et une grille de sous-graphiques
    fig, ax = plt.subplots(figsize=(12, 15))

    # Tracé les barres horizontales
    nb_lignenull.plot.barh(ax=ax, color='b', legend=False, grid=True)

    # Ajout des titres et des étiquettes d'axe
    ax.set_title("Pourcentage de valeurs nulles par série")
    ax.set_xlabel("Pourcentage de valeurs nulles")
    ax.set_ylabel("Séries")

    # Afficher le graphique
    plt.show()

## Exploration du DataFrame foot_note
### Exploration globale

In [ ]:
foot_note.info

In [ ]:
# Vérification du nombre de lignes dupliquées
foot_note.duplicated().sum()

In [ ]:
# Vérification du type des colonnes
foot_note.dtypes

In [ ]:
foot_note["CountryCode"].nunique()

In [ ]:
foot_note["SeriesCode"].nunique()

In [ ]:
foot_note["Year"].nunique()

### Visuels des données manquantes

In [ ]:
# Affichage des données d'un échantillon de 1000 lignes
msno.bar(foot_note.sample(1000))

In [ ]:
# Affichage des données présentes par ligne pour un échantillon de 500 lignes 
msno.matrix(foot_note.sample(500))

In [ ]:
# Calcul du nombre données manquantes
foot_note.isnull().sum()

## Exploration du DataFrame country_series
### Exploration globale

In [ ]:
country_series.info

In [ ]:
# Vérification du nombre de lignes dupliquées
country_series.duplicated().sum()

In [ ]:
# Vérification du type des colonnes
country_series.dtypes

In [ ]:
country_series["CountryCode"].nunique()

In [ ]:
country_series["SeriesCode"].nunique()

In [ ]:
list_codes_country_series = country_series["SeriesCode"].unique()
series.loc[series["Series Code"].isin(list_codes_country_series), "Indicator Name"]

### Visuels des données manquantes

In [ ]:
# Affichage des données d'un échantillon de 1000 lignes
msno.bar(country_series)

In [ ]:
# Affichage des données présentes par ligne pour un échantillon de 500 lignes 
msno.matrix(country_series)

In [ ]:
# Calcul du nombre données manquantes
country_series.isnull().sum()

# Création du Dataframe avec 4 indicateurs importants

In [ ]:
asked_data = data.loc[data["Indicator Name"].isin(indicateurs_precis)].copy()

## Visuel des données manquantes 

In [ ]:
# Calcul de la moyenne des valeurs manquantes par colonne
nb_colnull = asked_data.isnull().mean() * 100

# Création de la figure avec 2 sous-graphique
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))

# Création du graphique pour les colonnes de 1970 à 2017
nb_colnull.loc["1970":"2017"].plot(
    kind="bar",
    ax=ax1,
    title="Pourcentage de valeurs vides par colonne",
    xlabel="Nom des Colonnes",
    ylabel="Pourcentage")

# Ajout des valeurs en haut des colonnes
for i, value in enumerate(nb_colnull.loc["1970":"2017"]):
    ax1.annotate(f"{int(value)}", (i, value), ha="center", va="bottom")
    
# Création du graphique pour les colonnes de 2020 à 2100
nb_colnull.loc["2020":].plot(kind="bar",
                             ax=ax2,
                             xlabel="Nom des Colonnes",
                             ylabel="Pourcentage")

# Ajout des valeurs en haut des colonnes
for i, value in enumerate(nb_colnull.loc["2020":]):
    ax2.annotate(f"{int(value)}", (i, value), ha="center", va="bottom")
    
# Ajuster l'espacement entre les sous-graphiques
plt.tight_layout()

# Afficher le graphique
plt.show()

In [ ]:
round(nb_colnull.mean(), 2)

### Suppression des colonnes vides

In [ ]:
# On garde les colonnes de la première jusqu'à celle de 2016
asked_data = asked_data.loc[:, :"2016"]

# On récupère la lsite des colonnes de dates avec les données numériques
liste_dates = list(asked_data.loc[:, "1970":].columns)
asked_data.head()

### Visuel des données manquantes par pays

In [ ]:
# Calcul de la moyenne des données vides par ligne
nb_lignenull = asked_data.set_index("Country Name").isnull().mean(axis=1) * 100

# Regroupement par pays pour avoir la moyenne des données vide par pays
grouped_data = nb_lignenull.reset_index().groupby("Country Name").mean()

# Créer une figure et une grille de sous-graphiques
fig, ax = plt.subplots(figsize=(12, 50))

# Tracer les barres horizontales
grouped_data.plot.barh(ax=ax, color='b', legend=False, grid=True)

# Ajouter des titres et des étiquettes d'axe
ax.set_title("Pourcentage de valeurs nulles par pays")
ax.set_xlabel("Pourcentage de valeurs nulles")
ax.set_ylabel("Pays")

# Afficher le graphique
plt.show()

## Tri des pays avec suffisamment de données

In [ ]:
# Récupération des pays avec moins de 55% de données vide
pays_a_garder = grouped_data[grouped_data < 55].dropna().index

# On garde les données pour ces pays
asked_data = asked_data[asked_data["Country Name"].isin(pays_a_garder)]
asked_data.reset_index(drop=True, inplace=True)

asked_data.shape

In [ ]:
# Nombre de pays restants après le tri
asked_data["Country Name"].nunique()

## Vérification des valeurs abérrantes

In [ ]:
# Vérification de l'accès à internet
data_internet = asked_data[asked_data["Indicator Name"] == "Internet users (per 100 people)"]
data_internet[liste_dates].max()

In [ ]:
list_countries = asked_data["Country Name"].unique()

verif_data = asked_data.copy()

# Changement du nom des indicateurs dans le DataFrame
for indic in indicateurs_precis:
    nouvel_indic = indic.replace(', both sexes (number)', '')
    verif_data.loc[verif_data["Indicator Name"] == indic,
                   "Indicator Name"] = nouvel_indic
    
# Récupération du nom des nouveaux indicateurs
nouveaux_indic = verif_data["Indicator Name"].unique()

# Boucle sur la liste des pays
for i, country in enumerate(list_countries):
    
    # Création de la figure
    fig, axes = plt.subplots(1, 4, figsize=(16, 6))
    
    # Boucle sur les indicateurs
    for j, indic in enumerate(nouveaux_indic):
        
        # Récupération des informations utiles
        frame = verif_data[(verif_data["Country Name"] == country) & (
            verif_data["Indicator Name"] == indic)].set_index("Country Name")
        
        # Création des boxplot par indicateurs après transposition du DataFrame
        axes[j].boxplot(frame[liste_dates].T)
        
        # Ajout du titre du graphique
        axes[j].set_title(indic)
        
        # Ajout du nom du pays en abscisse
        axes[j].set_xticklabels([country])

    # Ajout d'un sous-titre
    plt.suptitle(country, fontsize=16)
    
    # Ajustage des marges
    plt.tight_layout()
    
    # Affichage du graphique
    plt.show()

In [ ]:
# Boucle sur la liste des pays
for i, country in enumerate(list_countries):
    
    # Création de la figure
    fig, axes = plt.subplots(1, 4, figsize=(16, 6))
    
    # Récupération des informations utiles
    frame = verif_data[verif_data["Country Name"] == country].set_index("Indicator Name")
    
    # Transposition des données
    frame = frame[liste_dates].T
    
    x = np.arange(2)
    
    #Boucle sur les indicateurs
    for j, indic in enumerate(nouveaux_indic):
        
        if not pd.isna(frame[indic].max()):
            # Récupération des maximum et minimum
            y = [frame[indic].max(), frame[indic].min()]

            # Affichage des 2 barres de 2 couleurs
            axes[j].bar(x=x, height=y, color=["b", "orange"])

            axes[j].set_xticks(x)
            # Ajout des labels des abscisses
            axes[j].set_xticklabels(["Max", "Min"], rotation=45, ha='right')

            # Ajout du titre
            axes[j].set_title(indic)

            #Ajout des valeurs au dessus des barres
            for k, value in enumerate(y):
                    axes[j].annotate(str(round(value)), xy=(x[k], value), xytext=(x[k], value+0.5),
                                     ha='center', va='bottom', fontsize=10)

        
    # Ajout d'un sous-titre 
    plt.suptitle(country, fontsize=16)
    
    # Ajustage des marges
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    # Affichage du graphique
    plt.show()

## Ajout de colonnes (Dernière valeur, Max, Min, Moy, Med, Ecart-type)

In [ ]:
# Ajout de colonnes avec ces données : Max, Min, Moyenne, Médiane et Ecart-type
asked_data["Max"] = asked_data[liste_dates].max(axis=1)
asked_data["Min"] = asked_data[liste_dates].min(axis=1)
asked_data["Moyenne"] = asked_data[liste_dates].mean(axis=1)
asked_data["Mediane"] = asked_data[liste_dates].median(axis=1)
asked_data["Ecart-type"] = asked_data[liste_dates].std(axis=1)

# Ajout des colonnes de dernières valeurs et de la colonne qui lui correspond
last_line_value(asked_data, liste_dates)

### Quelques Stats

In [ ]:
asked_data.loc[asked_data["Country Name"]=="World", ["Indicator Name", "Max", "Min"]]

In [ ]:
stats_data = asked_data.copy()
max_data = pd.DataFrame(columns=stats_data.columns)
min_data = pd.DataFrame(columns=stats_data.columns)

for indic in indicateurs_precis:
    indic_data = stats_data[stats_data["Indicator Name"] == indic]
    max_data = pd.concat([max_data, indic_data.sort_values("Max", ascending=False).head(15)])
    min_data = pd.concat([min_data, indic_data.sort_values("Min", ascending=True).head(10)])

max_data[["Country Name", "Indicator Name", "Max"]]

In [ ]:
min_data[["Country Name", "Indicator Name", "Min"]]

In [ ]:
regions_stats = stats_data.iloc[:99]

min_regions_data = pd.DataFrame(columns=stats_data.columns)

for indic in indicateurs_precis:
    indic_data = regions_stats[regions_stats["Indicator Name"] == indic]
    min_regions_data = pd.concat([min_regions_data, indic_data.sort_values("Min", ascending=True).head(10)])
    
min_regions_data[["Country Name", "Indicator Name", "Min"]]

# Visuels des données pour la France

In [ ]:
# Création du DataFrame avec les données pour la France
data_france = asked_data[asked_data["Country Name"] == "France"].copy()
data_france.head()

## Tendances

In [ ]:
# Création des graphs de tendance des données par indicateur
graphs(data_france, liste_dates)

## Tendances avec Dataframe transposé

In [ ]:
# Transposition du DataFrame avec les indicateurs en index
transposed_france = data_france.set_index("Indicator Name").loc[:, "1970":"2016"].T

# Affichage des tendances des données
transposed_graphs(transposed_france)

## Tendance avec les données interpolées

In [ ]:
#Changement de nom des colonnes
nouvelles_colonnes = [
    colonne.replace(', both sexes (number)', '')
    for colonne in transposed_france.columns
]

# Changement du nom des colonnes dans le DataFrame
transposed_france.rename(columns=dict(
    zip(transposed_france.columns, nouvelles_colonnes)),
                         inplace=True)

In [ ]:
# Interpolation des données
interpolated_france = transposed_france.interpolate()

# Affichage des tendances avec une couleur différente pour les données interpolées
interpolated_graphs(transposed_france, interpolated_france)

## Corrélation
### Distribution des valeurs

In [ ]:
# Affichage de la distribution des valeurs par indicateurs
for indic in indicateurs_precis:
    frame = data_france.loc[data_france["Indicator Name"] == indic, liste_dates].T
    
    plt.hist(frame)
    
    # Afficher le graphique
    plt.show()

### Corrélation

In [ ]:
# test de la correlation entre les indicateurs pour la France
correlation_france = transposed_france.corr()

# Affichage d'une heatmap avec la corrélation
ax = sns.heatmap(correlation_france,
                 vmin=-1,
                 vmax=1,
                 center=0,
                 cmap="coolwarm",
                 square=True)

# Mise en forme des labels des abscisses
ax.set_xticklabels(ax.get_xticklabels(),
                   rotation=45,
                   horizontalalignment="right")

# Ajout su titre
ax.set_title("Corrélation entre les indicateurs pour la France")

### Relation entre le nombre d'étudiants en secondaire et en tertiaire

In [ ]:
# Affichage de la relation entre le nombre d'étudiants en secondaire et tertiaire
# Avec une couleur différente en fonction de l'accès à Internet
interpolated_france.plot.scatter(1,
                                 2,
                                 c=3,
                                 s=interpolated_france.iloc[:, 3] * 3,
                                 colormap="plasma",
                                 title="Relation entre le nombre d'étudiants en secondaire et en tertiaire en France",
                                 alpha=0.5)

In [ ]:
# Affichage de la droite de régression linéaire pour ces mêmes données
sns.regplot(
    x="Enrolment in secondary education",
    y="Enrolment in tertiary education, all programmes",
    data=interpolated_france,
    marker="x",
    color="g"
).set(
    title=
    "Droite de Régression linéaire de la relation entre le secondaire et le tertiaire en France"
)

# Afficher le graphique
plt.show()

In [ ]:
# Affichage de la droite de régression polynomiale pour ces mêmes données
sns.lmplot(
    x="Enrolment in secondary education",
    y="Enrolment in tertiary education, all programmes",
    data=interpolated_france,
    order=2,
).set(
    title=
    "Droite de Régression polynomiale d'ordre 2 en France"
)

# Afficher le graphique
plt.show()

### Relation en le nombre d'étudiant en post-secondaire et l'accès à internet

In [ ]:
# Affichage de la relation entre le nombre d'étudiants en psot-secondaire et l'accès à internet
interpolated_france.plot.scatter(
    0,
    3,
    alpha=0.5,
    title=
    "Relation en le nombre d'étudiants en post-secondaire et l'accès à Internet en France"
)

In [ ]:
# Affichage de la droite de régression linéaire pour ces mêmes données
sns.regplot(x="Enrolment in post-secondary non-tertiary education",
            y="Internet users (per 100 people)",
            data=interpolated_france,
            marker="x",
            color="g").set(
    title=
    "Droite de Régression linéaire pour la France"
)
plt.show()

In [ ]:
# Affichage de la droite de régression polynomiale pour ces mêmes données
sns.lmplot(
    x="Enrolment in post-secondary non-tertiary education",
    y="Internet users (per 100 people)",
    data=interpolated_france,
    order=2,
).set(
    title=
    "Droite de Régression polynomiale d'ordre 2 pour la France"
)
plt.show()

# Corrélation avec les Etats-Unis

In [ ]:
# Récupération des données pour les Etats-Unis
data_usa = asked_data[asked_data["Country Code"] == "USA"]

# Tranposition des données
transposed_usa = data_usa.set_index("Indicator Name").loc[:, "1970":"2016"].T

# Interpolation des données
interpolated_usa = transposed_usa.interpolate()

#Calcul des corrélations des données
correlation_usa = interpolated_usa.corr()

# Affichaige de la Heatmap de ces corrélations
ax = sns.heatmap(correlation_france,
                 vmin=-1,
                 vmax=1,
                 center=0,
                 cmap="coolwarm",
                 square=True)

# Mise en forme des labels de l'axes des abscisses
ax.set_xticklabels(ax.get_xticklabels(),
                   rotation=45,
                   horizontalalignment="right")

# Traitement des données vides
## Internet users

In [ ]:
# Récupération des données pour l'accès à internet pour les années de 1970 à 1989
data_internet = asked_data[
    asked_data["Indicator Name"] ==
    "Internet users (per 100 people)"].loc[:, "1970":"1989"]

# Vérification des données manquantes
data_internet.isnull().sum()

In [ ]:
# Affichage des doonées les plus remplies
data_internet[~data_internet["1989"].isnull()]

In [ ]:
# remplissage des donénes de 1970 à 1989 à 0
asked_data.loc[data_internet.index, "1970":"1989"] = 0

In [ ]:
# Vérification des doonées manquantes
asked_data.isnull().mean()

## Interpolation par Indicateurs
### Avec la fonction pandas.interpolate()

In [ ]:
# Séparation des données par indicateurs avec transposition
# Données d'accès à internet
data_internet = asked_data[
    asked_data["Indicator Name"] ==
    "Internet users (per 100 people)"].loc[:, liste_dates].T

# Données des étudiants en secondaire
data_secondary = asked_data[
    asked_data["Indicator Name"] ==
    "Enrolment in secondary education, both sexes (number)"].loc[:,
                                                                 liste_dates].T

# Données des étudiants en post-secondaire
data_p_secondary = asked_data[
    asked_data["Indicator Name"] ==
    "Enrolment in post-secondary non-tertiary education, both sexes (number)"].loc[:,
                                                                                   liste_dates].T

# Données des étudiants en tertiaire
data_tertiary = asked_data[
    asked_data["Indicator Name"] ==
    "Enrolment in tertiary education, all programmes, both sexes (number)"].loc[:,
                                                                                liste_dates].T

# Interpolation des données par indicateurs
data_internet = data_internet.interpolate()
data_secondary = data_secondary.interpolate()
data_p_secondary = data_p_secondary.interpolate()
data_tertiary = data_tertiary.interpolate()

### Avec SimpleImputer pour "Internet users", "secondary" et "tertiary"

In [ ]:
# Création du SimpleImputer qui va remplir les valeurs NaN par la moyenne
imp_mean = SimpleImputer(missing_values=np.nan, strategy="mean")

# mise à jour des DataFrame après le remplissage
# Données de l'accès à Internet
data_internet = pd.DataFrame(imp_mean.fit_transform(data_internet),
                             columns=data_internet.columns,
                             index=data_internet.index).T

# Données des étudiants en secondaie
data_secondary = pd.DataFrame(imp_mean.fit_transform(data_secondary),
                              columns=data_secondary.columns,
                              index=data_secondary.index).T

# Données des étudiants en tertiaire
data_tertiary = pd.DataFrame(imp_mean.fit_transform(data_tertiary),
                             columns=data_tertiary.columns,
                             index=data_tertiary.index).T

# Il y a trop de valeurs manquantes pour utiliser le SimpleImputer
data_p_secondary = data_p_secondary.T

### Mise à jour du Dataframe Asked_data

In [ ]:
# On met à jour les données avec celles interpolées et imputées

# Données sur l'accès Internet
asked_data.loc[asked_data["Indicator Name"] ==
               "Internet users (per 100 people)", liste_dates] = data_internet

# Données des étudiants en secondaire
asked_data.loc[asked_data["Indicator Name"] ==
               "Enrolment in secondary education, both sexes (number)",
               liste_dates] = data_secondary

# Données des étudiants en port-secondaire
asked_data.loc[
    asked_data["Indicator Name"] ==
    "Enrolment in post-secondary non-tertiary education, both sexes (number)",
    liste_dates] = data_p_secondary

#Doonées des étudiants en tertiaire
asked_data.loc[
    asked_data["Indicator Name"] ==
    "Enrolment in tertiary education, all programmes, both sexes (number)",
    liste_dates] = data_tertiary

## Remplissage des données pour le post-secondaire

In [ ]:
# Boucle sur toutes les lignes du DataFrame contenant l'indicateur des étudiants en post-secondaire
for i in range(0, len(asked_data), 4):

    # Récupération des données des étudiants en post_secondaire
    x = asked_data.loc[i, "2000":"2013"]

    # Récupération des données des étudiants en tertiaire
    y = asked_data.loc[i + 2, "2000":"2013"]

    # Calcul de la moyenne du rapport entre ces deux indices
    moy = (x / y).mean()

    # Récupération des colonnes de données
    inter_colonnes = list(asked_data.loc[:, "1970":"2016"].columns)

    # Inversement de l'ordre des colonnes
    inter_colonnes.sort(reverse=True)

    # Boucle sur chaqeu colonne
    for colonne in inter_colonnes:

        # Vérification si la valeur est nulle
        if pd.isna(asked_data.loc[i, colonne]):
            # Ajout de la valeur de la moyenne de la ligne
            asked_data.loc[i, colonne] = asked_data.loc[i + 2, colonne] * moy

        # Vérification si une valeur a été ajoutée
        if pd.isna(asked_data.loc[i, colonne]):
            # S'il n'y a pas de moyenne de ligne, on rempli par la moyenne de l'indicateur
            asked_data.loc[i, colonne] = asked_data.loc[
                asked_data["Indicator Name"] ==
                "Enrolment in post-secondary non-tertiary education, both sexes (number)",
                colonne].mean()

In [ ]:
asked_data.isnull().mean()

In [ ]:
# Si la valeur de Last Value est vide, on la remplace par celle de 2016
asked_data.loc[asked_data["Last Value"].isnull(),
               "Last Value"] = asked_data.loc[
                   asked_data["Last Value"].isnull(), "2016"]

# Si la valeur de Last Value Date est vide, on la remplace par 2016
asked_data.loc[asked_data["Last Value Date"].isnull(),
               "Last Value Date"] = 2016

## Shape final des données

In [ ]:
asked_data.shape

## Sauvegarde des données traitées

In [ ]:
# Sauvegarde des données ainsi filtrée et complétée
asked_data.to_csv("../Data/FilteredData.csv", index=False, sep=";")

# Indicateurs statistiques
## Séparation des donées par régions

In [ ]:
# Récupération de la liste des régions
liste_regions = list(country_data["Region"].unique())

# Suppression des valeurs nulles
liste_regions.remove(np.nan)

# Récupération des pays par région
liste_pays = []
for region in liste_regions:
    liste_pays.append(country_data.loc[country_data["Region"] == region,
                                  "Short Name"].unique())

# création de DataFrame pour chaque région et avec les régions
data_regions = asked_data[asked_data["Country Name"].isin(liste_regions)]
data_s_america = asked_data[asked_data["Country Name"].isin(liste_pays[0])]
data_s_asia = asked_data[asked_data["Country Name"].isin(liste_pays[1])]
data_s_africa = asked_data[asked_data["Country Name"].isin(liste_pays[2])]
data_europe = asked_data[asked_data["Country Name"].isin(liste_pays[3])]
data_n_africa = asked_data[asked_data["Country Name"].isin(liste_pays[4])]
data_e_asia = asked_data[asked_data["Country Name"].isin(liste_pays[5])]
data_n_america = asked_data[asked_data["Country Name"].isin(liste_pays[6])]

# Création d'une liste contenant les noms des pays sélectionnés par région
liste_pays = [
    list(data_s_america["Country Name"].unique()),
    list(data_s_asia["Country Name"].unique()),
    list(data_s_africa["Country Name"].unique()),
    list(data_europe["Country Name"].unique()),
    list(data_n_africa["Country Name"].unique()),
    list(data_e_asia["Country Name"].unique()),
    list(data_n_america["Country Name"].unique()),
]

# Affichage de chaque région, avec le nombre de pays et le nom des pays
print(*(
    f"{i+1}. Pays de {liste_regions[i]} ({len(liste_pays[i])} pays) :\n\n {liste_pays[i]}"
    for i in range(0, len(liste_regions))),
      sep="\n\n\n")

In [ ]:
# Création d'une liste avec les dataframes
liste_frames = [
    data_regions, data_s_america, data_s_asia, data_s_africa, data_europe,
    data_n_africa, data_e_asia, data_n_america
]

# Ajout du nom du premier DataFrame contenant toutes les régions
liste_noms_regions = liste_regions.copy()
liste_noms_regions.insert(0, "World regions")

# Boucle sur chacun des DataFrame
for i, frame in enumerate(liste_frames):
    
    # Changement de l'index pour l'affichage sur le graphique
    frame = frame.set_index("Country Name")
    
    # Boucle sur chacun des indicateurs
    for indic in indicateurs_precis:
        
        # Récupération des données utiles 
        frame_indic = frame.loc[frame["Indicator Name"] == indic, "1970":"2016"]
        
        # Récupération de la longueur du Dataframe pour adapter la taille du graphique
        len_frame = len(frame_indic)
        
        # Création de la figure avec une taille adaptée
        plt.figure(figsize=(12, len_frame))
        
        # Transposition des données et création du boxplot
        frame_indic.T.boxplot(vert=False)
        
        # Création du label pour les abscisses différents selon l'indicateur
        if indic == "Internet users (per 100 people)":
            plt.xlabel("Pourcentage de personnes ayant accès à internet")
        else:
            plt.xlabel("Nombre d'étudiants")
            
        # Création du mabel pour les ordonnées
        plt.ylabel("Pays")
        
        # Ajout du titre
        plt.title(f"{indic} in {liste_noms_regions[i]}")
        
        # Affichage du graphique
        plt.show()

In [ ]:
data_regions[data_regions["Indicator Name"] == "Internet users (per 100 people)"].sort_values("Max",ascending=False)

In [ ]:
# Boucle sur chacun des DataFrame
for i, frame in enumerate(liste_frames):
    
    # Changement de l'index pour l'affichage sur le graphique
    frame = frame.set_index("Country Name")
    
    # Boucle sur chacun des indicateurs
    for indic in indicateurs_precis:
        
        # Récupération des données utiles 
        frame_indic = frame.loc[frame["Indicator Name"] == indic, "Moyenne": "Ecart-type"]
        
        # Récupération de la longueur du Dataframe pour adapter la taille du graphique
        len_frame = len(frame_indic)
        
        # Création de la figure avec une taille adaptée
        plt.figure(figsize=(len_frame, 8))
        
        # Transposition des données et création du boxplot
        frame_indic.plot.bar(color=["b","r","g"])
        
        # Création du label pour les abscisses différents selon l'indicateur
        if indic == "Internet users (per 100 people)":
            plt.ylabel("Pourcentage de personnes ayant accès à internet")
        else:
            plt.ylabel("Nombre d'étudiants")
            
        # Création du mabel pour les ordonnées
        plt.xlabel("Pays")
        
        # Rotation et alignement des noms des pays
        plt.xticks(rotation=45, ha="right", va="top")
        
        # ajout de la grille
        plt.grid(True, alpha=0.8)
        
        # Ajout du titre
        plt.title(f"{indic} in {liste_noms_regions[i]}")
        
        # Affichage du graphique
        plt.show()

# Création du Score

In [ ]:
# Création d'une liste des indicateurs représentés par des nombres d'étudiants
number_indic = [
    "Enrolment in post-secondary non-tertiary education, both sexes (number)",
    "Enrolment in secondary education, both sexes (number)",
    "Enrolment in tertiary education, all programmes, both sexes (number)",
]

#Tri des données sur ces indicateurs
number_data = asked_data[asked_data["Indicator Name"].isin(number_indic)]

# Regroupement par pays en ajoutant le nombre total d'étudiants
grouped_number = number_data.groupby("Country Name").sum().loc[:, "1970":]

# Tri des données pour l'indicateur d'accès à internet
internet_data = asked_data.loc[asked_data["Indicator Name"] ==
                               "Internet users (per 100 people)"]

# Transposition des DataFrame
grouped_number = grouped_number.T
internet_data = internet_data.set_index("Country Name").loc[:, "1970":].T

# Création d'un DataFrame de score en multipliant le nombre d'étudiant par le pourcentage d'accès à internet
score_data = grouped_number * (internet_data / 100)

# Retransposition des données et reset de l'index
score_data = score_data.T.reset_index()

In [ ]:
# Ajout d'une colonne contenant le pourcentage d'évolution du nombre d'étudiants 
# avec accès à internet sur les 6 dernière années

score_data["Evolution 6ans"] = (score_data["2016"] -
                                score_data["2010"]) / score_data["2010"]

# Même chose mais sur 2 ans
score_data["Evolution 2ans"] = (score_data["2016"] -
                                score_data["2014"]) / score_data["2014"]

# Mise à jour de la colonne Max
score_data["Max"] = score_data.loc[:, "1970":"2016"].max(axis=1)

In [ ]:
# Exploration de la colonne Last Value
score_data["Last Value"].describe()

In [ ]:
# Exploration de la colonne Evolution 6ans
score_data["Evolution 6ans"].describe()

In [ ]:
# Vérification des valeurs maximales
score_data.loc[score_data["Evolution 6ans"] > 40]

In [ ]:
# Exploration de la colonne Evolution 2ans
score_data["Evolution 2ans"].describe()

## Séparation des données par régions

In [ ]:
# création de DataFrame pour chaque région et avec les régions
score_regions = score_data[score_data["Country Name"].isin(liste_regions)]
score_s_america = score_data[score_data["Country Name"].isin(liste_pays[0])]
score_s_asia = score_data[score_data["Country Name"].isin(liste_pays[1])]
score_s_africa = score_data[score_data["Country Name"].isin(liste_pays[2])]
score_europe = score_data[score_data["Country Name"].isin(liste_pays[3])]
score_n_africa = score_data[score_data["Country Name"].isin(liste_pays[4])]
score_e_asia = score_data[score_data["Country Name"].isin(liste_pays[5])]
score_n_america = score_data[score_data["Country Name"].isin(liste_pays[6])]

In [ ]:
liste_noms_regions

## Représentation du score

In [ ]:
# Création d'un DataFrame sans la Chine et le Myanmar qui étendent trop le graphique
score_e_asia_mid = score_e_asia[~(
    score_e_asia["Country Name"].isin(["China", "Myanmar"]))]

# Création d'une liste avec tous les DataFrame de score
liste_frames = [
    score_regions, score_s_america, score_s_asia, score_s_africa, score_europe,
    score_n_africa, score_e_asia, score_e_asia_mid, score_n_america
]

# Création d'une liste avec les noms à afficher en titre de graphique
liste_regions_fr = [
    "Régions du Monde", "Amérique Latine et Caraïbe", "Asie du Sud",
    "Afrique Sub-Saharienne", "Europe et Asie Centrale",
    "Moyen Orient et Afrique du Nord", "Asie de l'Est et Pacifique",
    "Asie de l'Est et Pacifique sans la Chine et le Myanmar",
    "Amérique du Nord"
]

# Création de trois DataFrame vides pour les rmplir avec les pays qui ont certains scores.
top_pays = pd.DataFrame(columns=score_data.columns)
pays_top_last_value = pd.DataFrame(columns=score_data.columns)
pays_top_evol = pd.DataFrame(columns=score_data.columns)

# Boucle sur tous les DataFrame de notre liste
for i, frame in enumerate(liste_frames):
    # Création de variables contenant les médianes des colonnes "Last Value" et "Evolution 6ans"
    med_last_value = frame["Last Value"].median()
    med_evol_6 = frame["Evolution 6ans"].median()
    max_last_value = frame["Last Value"].max()

    # Création d'une liste contenant les pays avec les plus hautes "Last Value" et "Evolution 6 ans"
    top_pays_names = pd.concat([
        frame.sort_values(["Last Value"], ascending=False).head(6),
        frame.sort_values(["Evolution 6ans"], ascending=False).head(6)
    ])

    # Création de liste de pays avec des valeurs supérieures aux deux médianes
    top_pays = pd.concat([
        top_pays,
        frame.loc[(frame["Last Value"] >= med_last_value)
                  & (frame["Evolution 6ans"] >= med_evol_6)].sort_values(
                      "Last Value", ascending=False)
    ])

    # Création de liste avec les pays avec une valeur supérieure pour la colonne "Last Value"
    pays_top_last_value = pd.concat([
        pays_top_last_value,
        frame.loc[(frame["Last Value"] >= med_last_value)
                  & ~(frame["Country Name"].isin(top_pays["Country Name"]))].
        sort_values("Last Value", ascending=False)
    ])

    # Création de liste avec les pays avec une valeur supérieure pour la colonne "Evolution 6ans"
    pays_top_evol = pd.concat([
        pays_top_evol,
        frame.loc[(frame["Evolution 6ans"] >= med_evol_6)
                  & ~(frame["Country Name"].isin(top_pays["Country Name"]))].
        sort_values("Evolution 6ans", ascending=False)
    ])

    # Création d'un graphique représentant la valeur de la dernière valeur en fonction de l'évolution
    plt.figure(figsize=(15, 8))
    scatter = plt.scatter(x=frame["Evolution 6ans"] * 100,
                          y=frame["Last Value"],
                          c=frame["Evolution 2ans"] * 100)

    # Ajout de la légende pour la couleur
    cbar = plt.colorbar(scatter)
    cbar.set_label("Évolution sur 2 ans (%)")

    # Ajout des noms des pays sur les points du graphique
    for j, row in top_pays_names.iterrows():
        if (j % 2) == 0:
            va = "bottom"
            x = max_last_value * 0.01
        else:
            va = "top"
            x = -max_last_value * 0.01
        plt.text(row["Evolution 6ans"] * 100,
                 row["Last Value"] + x,
                 row["Country Name"],
                 ha="center",
                 va=va)

    # Ajout des lignes pour les valeurs des médianes
    plt.axvline(x=med_evol_6 * 100, color="gray", linestyle="--", alpha=0.8)
    plt.axhline(y=med_last_value, color="gray", linestyle="--", alpha=0.8)

    # Ajout des labels et titre
    plt.xlabel("Evolution sur 6 ans (%)")
    plt.ylabel("Nombre d'étudiants avec accès à Internet")
    plt.title(
        f"Nombre d'étudiants potentiels en fonction de l'évolution de leur nombre en {liste_regions_fr[i]}"
    )

    # Affichage des graphiques
    plt.show()

In [ ]:
top_pays[["Country Name", "Last Value", "Evolution 6ans", "Evolution 2ans"]].sort_values("Last Value", ascending=False)

In [ ]:
pays_top_last_value[[
    "Country Name", "Last Value", "Evolution 6ans", "Evolution 2ans"
]].sort_values("Last Value", ascending=False)

In [ ]:
pays_top_evol[[
    "Country Name", "Last Value", "Evolution 6ans", "Evolution 2ans"
]].sort_values("Evolution 6ans", ascending=False)

# Autres informations
## Définition des indicateurs

In [ ]:
# Récupération des longues définitions des indicateurs choisis
data_indicateurs = series.loc[
    series["Indicator Name"].isin(indicateurs_precis),
    "Long definition"].reset_index(drop=True)

# Affichage des longues définitions
print(*(f"{indicateurs_precis[i]} : \n\n{data_indicateurs.iloc[i]}"
        for i in range(0, len(data_indicateurs))),
      sep="\n\n\n")

## Méthode de récolte des données

In [ ]:
# Récupération des codes des pays
liste_code_pays = list(asked_data["Country Code"].unique())

# Récupération des codes des indicateurs
liste_code_indic = series.loc[
    series["Indicator Name"].isin(indicateurs_precis), "Series Code"]

# Récupération des données pour nos codes pays et indicateurs
data_foot_note = foot_note[
    (foot_note["CountryCode"].isin(liste_code_pays))
    & (foot_note["SeriesCode"].isin(liste_code_indic))].reset_index(drop=True).copy()

# Création des colonnes avec les noms des pays et des indicateurs
data_foot_note["Country Name"] = np.nan
data_foot_note["Indicator Name"] = np.nan

# Boucle sur toutes les lignes du tableau
for i in range(data_foot_note.shape[0]):
    
    # Récupération du code de pays
    code = data_foot_note.loc[i, "CountryCode"]
    
    # Récupération du code des indicateurs
    indic = data_foot_note.loc[i, "SeriesCode"]
    
    # récupération des informations de la ligne
    ligne = data_foot_note.loc[i].copy()
    
    # Mise à jour des colonnes "Country Name" et "Indicator Name"
    ligne["Country Name"] = country_data.loc[country_data["Country Code"] ==
                                                 code, "Short Name"]
    ligne["Indicator Name"] = series.loc[series["Series Code"] ==
                                                  indic, "Indicator Name"]
    
    # Mise à jour du DataFrame principal
    data_foot_note.loc[i] = ligne

data_foot_note.head()

In [ ]:
# Nombre de descriptions récupérées
data_foot_note["DESCRIPTION"].nunique()

In [ ]:
# Récupération des informations pour les pays et indicateurs choisis
data_series = country_series[
    (country_series["CountryCode"].isin(liste_code_pays))
    & (country_series["SeriesCode"].isin(liste_code_indic))].reset_index(drop=True).copy()

# Format des données combinées
data_series.shape

In [ ]:
# Récupération des informations pour les pays choisis
data_series_countries = country_series[
    country_series["CountryCode"].isin(liste_code_pays)].reset_index(drop=True).copy()

# Récupération des informations pour les indicateurs choisis
data_series_indic = country_series[
    country_series["SeriesCode"].isin(liste_code_indic)].reset_index(drop=True).copy()

# Format des données des pays
data_series_countries.shape

In [ ]:
# Format des données des indicateurs
data_series_indic.shape